# Download detectron2

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# Chech torch and detectron2
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Register dataset

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Ensure paths have forward slashes
train_image_path = '/kaggle/input/main-datasets/main_datasets/nighttime/nighttime_fold_0/train/image'
train_json_path = '/kaggle/input/main-datasets/main_datasets/nighttime/nighttime_fold_0/train/image/annotations.json'

# Unregister existing datasets and metadata if they were registered previously
MetadataCatalog.pop("my_train", None)
DatasetCatalog.pop("my_train", None)

# Register the datasets
register_coco_instances('my_train', {}, train_json_path, train_image_path)

# Declare metadata and load dataset dicts
train_metadata = MetadataCatalog.get("my_train")
train_dataset_dicts = DatasetCatalog.get("my_train")


# Training

In [ ]:
# Config the training proccess
# Now before we train we need to config
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = '/kaggle/working/output'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2  
cfg.SOLVER.BASE_LR = 0.001 
cfg.SOLVER.MAX_ITER = 19389   
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) 

# train
trainer.train()